----------------------------------------IMPORT-----------------------------------------

In [ ]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Agent_Tools import poem_tools, weather_tools, sentiment_tools
from Agents import Coordinator_Agent, Flight_Agent, Hotel_Agent, Travel_Agent
from Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from memory.memory import memory_manager
from memory.long_term_memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os



e:\Multi Agent\Agent_Tools\poem_tools.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"
c:\Users\minhv\anaconda3\envs\pytorch-env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


----------------------------------------IMPORT-----------------------------------------

----------------------------------------CLASS, MODEL-----------------------------------------

In [ ]:
load_dotenv()

llm = ChatOllama(model="llama3:8b")

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]

prompt = hub.pull("hwchase17/react")

----------------------------------------CLASS, MODEL-----------------------------------------

----------------------------------------LANGCHAIN TOOLS-----------------------------------------

In [ ]:
weatherllm_agent = create_react_agent(llm, [weather_tools.weatherllm_tool], prompt)
weatherllm_agent_executor = AgentExecutor(agent=weatherllm_agent, tools=[weather_tools.weatherllm_tool], verbose=True, handle_parsing_errors=True)

def call_weatherllm_agent(state: AgentState):
    result = weatherllm_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
    return {"messages": [f"Thời tiết tại: {result['output']}"]}

In [ ]:
sentiment_agent = create_react_agent(llm, [sentiment_tools.sentiment_tool], prompt)
sentiment_agent_executor = AgentExecutor(agent=sentiment_agent, tools=[sentiment_tools.sentiment_tool], verbose=True, handle_parsing_errors=True)

def call_sentiment_agent(state: AgentState):
    result = sentiment_agent_executor.invoke({"input": f"Phân tích cảm xúc của đoạn text sau: {state['input']}"})
    return {"messages": [f"Phân tích cảm xúc: {result['output']}"]}

In [ ]:
poet_agent = create_react_agent(llm, [poem_tools.poem_tool], prompt)
poet_agent_executor = AgentExecutor(agent=poet_agent, tools=[poem_tools.poem_tool], verbose=True, handle_parsing_errors=True)
def call_poet_agent(state: AgentState):
    result = poet_agent_executor.invoke({"input": f"Hãy viết một bài thơ về: {state['input']}"})
    return {"messages": [f"Bài thơ: {result['output']}"]}

In [ ]:
# weatherapi_agent = create_react_agent(llm, [weather_tools.weatherapi_tool], prompt)
# weatherapi_agent_executor = AgentExecutor(agent=weatherapi_agent, tools=[weather_tools.weatherapi_tool], verbose=True, handle_parsing_errors=True)

# def call_weatherapi_agent(state: AgentState):
#     result = weatherapi_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
#     return {"messages": [f"Thời tiết tại: {result['output']}"]}

----------------------------------------LANGCHAIN TOOLS-----------------------------------------

----------------------------------------LANGGRAPH-----------------------------------------

In [ ]:
graph_builder = StateGraph(AgentState)
graph_builder.add_node("weather_llm", call_weatherllm_agent)
graph_builder.set_entry_point("weather_llm")
graph_builder.add_edge("weather_llm", END)

graph = graph_builder.compile()

input_test3 = "Thời tiết Hồ Chí Minh hiện tại"

final_state = graph.invoke({"input": input_test3})
print("Ket qua cuoi cung:")
for message in final_state["messages"]:
    print("- ", message)



> Entering new AgentExecutor chain...
A Vietnamese question!

Thought: Hmm, I need to get the current weather for Ho Chi Minh City. I have a tool called WeatherLLM that can help me with this.

Action: Use WeatherLLM
Action Input: city="Ho Chi Minh"Use WeatherLLM is not a valid tool, try one of [WeatherLLM].

KeyboardInterrupt: 

In [ ]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("sentiment_analyzer", call_sentiment_agent)
graph_builder.add_node("poet", call_poet_agent)
graph_builder.add_node("weather_llm", call_weatherllm_agent)
#graph_builder.add_node("weather_api", call_weatherapi_agent)

graph_builder.set_entry_point("sentiment_analyzer")

graph_builder.add_edge("sentiment_analyzer", "poet")
graph_builder.add_edge("poet", "weather_llm")
#graph_builder.add_edge("weather_llm", "weather_api")
#graph_builder.add_edge("weather_api", END)
graph_builder.add_edge("weather_llm", END)


graph = graph_builder.compile()

input_test1 = "Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp."
input_test2 = "Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại."

final_state = graph.invoke({"input": input_test2})
print("Kết quả cuối cùng:")
for message in final_state["messages"]:
    print("- ", message)



> Entering new AgentExecutor chain...
Let's get started.

Thought: Since the goal is to analyze the sentiment of a given text, I should use the SentimentAnalyzer tool. The input question requires me to analyze the sentiment of a specific text.

Action: Use SentimentAnalyzer
Action Input: Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại.Use SentimentAnalyzer is not a valid tool, try one of [SentimentAnalyzer].

KeyboardInterrupt: 

----------------------------------------LANGGRAPH-----------------------------------------

----------------------------------------MULTI AGENT WITHOUT MEMORY-----------------------------------------

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)
workflow.add_node("coordinator", Coordinator_Agent_1.coordinator_agent)
workflow.add_node("travel_agent", Travel_Agent_1.travel_agent)
workflow.add_node("hotel_agent", Hotel_Agent_1.hotel_agent)
workflow.add_node("flight_agent", Flight_Agent_1.flight_agent)

workflow.set_entry_point("coordinator")

def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "coordinator")

workflow.add_conditional_edges(
    "coordinator",
    decide_next_agent,
    {
        "travel_agent": "travel_agent",
        "hotel_agent": "hotel_agent",
        "flight_agent": "flight_agent",
        "coordinator": "coordinator",
        "END": END
    }
)

workflow.add_edge("travel_agent", "coordinator")
workflow.add_edge("hotel_agent", "coordinator")
workflow.add_edge("flight_agent", "coordinator")

app = workflow.compile()

# HỆ THỐNG CHAT TỰ ĐỘNG - ĐÃ SỬA LỖI
def interactive_chat_system():
    print("🤖 HỆ THỐNG MULTI-AGENT DU LỊCH THÔNG MINH")
    print("=" * 60)
    print("Bạn có thể nhập bất kỳ yêu cầu nào về du lịch!")
    print("• Gõ 'thoát' để kết thúc")
    print("=" * 60)

    # Khởi tạo state với messages rỗng
    state = {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting"
    }

    # Chạy lần đầu để có lời chào
    try:
        output = app.invoke(state, config={"recursion_limit": 50})
        print(f"🤖: {output['messages'][-1].content}")
        state = output
    except Exception as e:
        print(f"🤖: Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?")
        state["messages"] = [AIMessage(content="Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?")]
        state["needs_user_input"] = True

    # Vòng lặp chat chính
    while True:
        try:
            # Nhập input từ user
            user_input = input("\n👤 Bạn: ").strip()

            if user_input.lower() in ['exit', 'quit', 'thoát', 'kết thúc']:
                print("🤖: Cảm ơn bạn! Hẹn gặp lại! 👋")
                break

            if not user_input:
                print("🤖: Bạn muốn hỏi gì về du lịch ạ?")
                continue

            # Thêm user input vào conversation
            new_messages = state['messages'] + [HumanMessage(content=user_input)]
            state['messages'] = new_messages
            state['needs_user_input'] = False

            # Xử lý với multi-agent system
            output = app.invoke(state, config={"recursion_limit": 50})

            print(f"👤: {user_input}")

            # Hiển thị response
            if output['messages']:
                last_message = output['messages'][-1]
                print(f"🤖: {last_message.content}")
            else:
                print("🤖: Tôi có thể giúp gì thêm cho bạn?")

            # Cập nhật state
            state = output

        except KeyboardInterrupt:
            print("\n🤖: Hẹn gặp lại bạn! 👋")
            break
        except Exception as e:
            print(f"🤖: Có lỗi xảy ra: {e}")
            print("🤖: Hãy thử lại với yêu cầu khác nhé!")
            # Reset state
            state = {
                "messages": [AIMessage(content="Xin lỗi, có lỗi xảy ra. Bạn muốn hỏi gì về du lịch?")],
                "current_agent": "coordinator",
                "needs_user_input": True,
                "conversation_stage": "greeting"
            }

# Chạy hệ thống
if __name__ == "__main__":
    interactive_chat_system()

🤖 HỆ THỐNG MULTI-AGENT DU LỊCH THÔNG MINH
Bạn có thể nhập bất kỳ yêu cầu nào về du lịch!
• Gõ 'thoát' để kết thúc
🤖: Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?
🤖: Cảm ơn bạn! Hẹn gặp lại! 👋


----------------------------------------MULTI AGENT WITHOUT MEMORY-----------------------------------------

----------------------------------------MULTI AGENT WITH SHORT-TERM MEMORY-----------------------------------------

In [ ]:


# Orchestrator: use Agents instead of Tools

def create_initial_state():
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }

AGENT_MAP = {
    "coordinator": Coordinator_Agent.coordinator_agent,
    "travel_agent": Travel_Agent.travel_agent,
    "hotel_agent": Hotel_Agent.hotel_agent,
    "flight_agent": Flight_Agent.flight_agent,
}

def run_multi_agent_chat():
    print("🤖 Multi-Agent with Short-Term Memory (Agents, no Tools)")
    print("=" * 60)
    print("Type 'exit' to quit, 'clear' to reset memory.")

    state = create_initial_state()

    while True:
        if not state["needs_user_input"]:
            step_fn = AGENT_MAP.get(state["current_agent"], Coordinator_Agent.coordinator_agent)
            state = step_fn(state)

            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")

            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.get_memory().add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Memory will persist for this session.")
            break

        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.get_memory().clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa memory. Bắt đầu lại.")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False

# To start the chat, run:
#run_multi_agent_chat()


----------------------------------------MULTI AGENT WITH SHORT-TERM MEMORY-----------------------------------------

----------------------------------------READ SHORT-TERM MEMORY-----------------------------------------

In [ ]:
mem = memory_manager.get_memory()  # hoặc memory_manager.get_memory("user_id")
print(mem.get_context_summary())
print(memory_manager.get_memory())
print(memory_manager.get_memory("user_id"))

Session: session_20250909_181022

ShortTermMemory(user_id='default_user', conversation_history=[], user_preferences={}, booking_info={}, last_updated=datetime.datetime(2025, 9, 9, 18, 10, 22, 905889), session_id='session_20250909_181022')
ShortTermMemory(user_id='user_id', conversation_history=[], user_preferences={}, booking_info={}, last_updated=datetime.datetime(2025, 9, 9, 18, 10, 22, 906894), session_id='session_20250909_181022')


In [ ]:
mem = memory_manager.get_memory()
for m in mem.conversation_history:
    if m["role"] in ["user", "assistant"]:
        print(f"{m['timestamp']} | {m['role']}: {m['content']}")

In [ ]:
for m in mem.get_recent_history(5):
    print(f"{m['timestamp']} | {m['role']}: {m['content']}")

----------------------------------------READ SHORT-TERM MEMORY-----------------------------------------

----------------------------------------MULTI AGENT WITH LONG-TERM MEMORY-----------------------------------------

In [ ]:


def run_multi_agent_chat():
    print("🤖 Multi-Agent with Short-Term & Long-Term Memory")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (short-term), 'clear_all' (both), 'mem_stats'")

    state = create_initial_state()

    while True:
        if not state["needs_user_input"]:
            step_fn = AGENT_MAP.get(state["current_agent"], Coordinator_Agent.coordinator_agent)
            state = step_fn(state)

            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")

            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Long-term memory has been saved.")
            break

        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term memory vẫn giữ.")
            continue

        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            long_term_memory.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue

        if user_input.lower() in ["mem_stats", "memory_stats"]:
            stats = long_term_memory.collection.count()
            print(f"📊 Long-term Memory: {stats} items stored")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        state["needs_user_input"] = False

#run_multi_agent_chat()

----------------------------------------MULTI AGENT WITH LONG-TERM MEMORY-----------------------------------------

In [ ]:
from memory.long_term_memory import long_term_memory
print("Total items:", long_term_memory.collection.count())

col = long_term_memory.collection
all_items = col.get(include=["documents","metadatas"])
for doc, meta in zip(all_items["documents"], all_items["metadatas"]):
    print(meta.get("timestamp"), meta.get("session_id"), meta.get("role"), ":", doc)

Total items: 17
2025-09-09T16:40:21.448706 session_20250909_164021 assistant : assistant: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
2025-09-09T16:40:27.869014 session_20250909_164021 user : user: thoát
2025-09-09T16:44:05.596010 session_20250909_164021 assistant : assistant: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
2025-09-09T16:44:34.084731 session_20250909_164021 user : user: tôi muốn lập kế hoạch cho chuyến du lịch sắp tới
2025-09-09T16:45:00.265882 session_20250909_164021 assistant : assistant: Phấn khởi! Tôi sẽ giúp bạn tạo lập một chương trình du lịch sáng tạo và hấp dẫn.

Để bắt đầu, có thể bạn muốn chia sẻ với tôi về sở thích và mong muốn của mình trong chuyến du lịch này? Ví dụ, bạn muốn đi đến đâu, trải nghiệm gì, thư giãn hay khám phá?

Hãy chia sẻ với tôi để tôi có thể giúp đỡ bạn lập kế hoạch du lịch phù hợp nhất!
2025-09-09T16:45:18.755736 session_20250909_164021 user : user: thoát
2025-0

In [ ]:
from memory.memory import memory_manager
sid = memory_manager.get_session_id()  # session hiện tại

items = long_term_memory.collection.get(
    where={"session_id": sid},
    include=["documents","metadatas"]
)
for doc, meta in zip(items["documents"], items["metadatas"]):
    print(meta.get("timestamp"), meta.get("role"), ":", doc)

In [ ]:
all_items = long_term_memory.collection.get(include=["metadatas"])
session_ids = sorted({m.get("session_id") for m in all_items["metadatas"] if m})
print(session_ids)

['session_20250909_164021']


In [ ]:
from langchain_community.chat_models.ollama import ChatOllama
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import json
from datetime import datetime
from typing import List, Dict, Any
import uuid

# Khởi tạo ChromaDB
chroma_client = chromadb.PersistentClient(
    path="./long_term_memory/",
    settings=Settings(anonymized_telemetry=False)
)

embedder = SentenceTransformer('keepitreal/vietnamese-sbert')
prompt = hub.pull("hwchase17/react")

class ChatContinuationSystem:
    def __init__(self):
        self.client = chroma_client
        self.embedder = embedder
        self.llm = llm

        # Tạo collections
        self.conv_collection = self.client.get_or_create_collection("conversation_history")
        self.context_collection = self.client.get_or_create_collection("conversation_context")

    def save_conversation(self, user_id: str, role: str, content: str, session_id: str):
        """Lưu conversation với session_id"""
        message_id = str(uuid.uuid4())
        embedding = self.embedder.encode(content).tolist()

        self.conv_collection.add(
            ids=message_id,
            embeddings=[embedding],
            documents=[content],
            metadatas={
                "user_id": user_id,
                "role": role,
                "session_id": session_id,
                "timestamp": datetime.now().isoformat()
            }
        )

    def save_conversation_context(self, user_id: str, session_id: str, context: Dict):
        """Lưu context của conversation để tiếp tục sau"""
        context_id = f"ctx_{user_id}_{session_id}"
        context_str = json.dumps(context, ensure_ascii=False)
        embedding = self.embedder.encode(context_str).tolist()

        self.context_collection.upsert(
            ids=[context_id],
            embeddings=[embedding],
            documents=[context_str],
            metadatas={
                "user_id": user_id,
                "session_id": session_id,
                "last_updated": datetime.now().isoformat(),
                "is_active": True
            }
        )

    def get_user_sessions(self, user_id: str) -> List[Dict]:
        """Lấy danh sách sessions của user"""
        results = self.context_collection.get(
            where={"user_id": user_id},
            include=["metadatas", "documents"]
        )

        sessions = []
        for i in range(len(results['ids'])):
            try:
                context_data = json.loads(results['documents'][i])
                sessions.append({
                    "session_id": results['metadatas'][i]['session_id'],
                    "last_updated": results['metadatas'][i]['last_updated'],
                    "context": context_data,
                    "is_active": results['metadatas'][i].get('is_active', False)
                })
            except:
                continue

        return sorted(sessions, key=lambda x: x['last_updated'], reverse=True)

    def load_conversation_context(self, user_id: str, session_id: str) -> Dict:
        """Load context của một session cụ thể"""
        context_id = f"ctx_{user_id}_{session_id}"

        results = self.context_collection.get(
            ids=[context_id],
            include=["documents", "metadatas"]
        )

        if results['ids']:
            context_data = json.loads(results['documents'][0])
            return context_data
        return {}

    def get_conversation_history(self, user_id: str, session_id: str, limit: int = 20) -> List[Dict]:
        """Lấy lịch sử hội thoại của session"""
        results = self.conv_collection.get(
            where={"user_id": user_id, "session_id": session_id},
            limit=limit,
            include=["metadatas", "documents"]
        )

        history = []
        for i in range(len(results['ids'])):
            history.append({
                "role": results['metadatas'][i]['role'],
                "content": results['documents'][i],
                "timestamp": results['metadatas'][i]['timestamp']
            })

        return sorted(history, key=lambda x: x['timestamp'])

    def continue_conversation(self, user_id: str, session_id: str, new_message: str) -> str:
        """Tiếp tục đoạn chat từ session cũ"""
        # Load context và history
        context = self.load_conversation_context(user_id, session_id)
        history = self.get_conversation_history(user_id, session_id, 10)

        # Tạo prompt với context cũ
        conversation_context = self._build_conversation_context(history, context)

        prompt = f"""
{conversation_context}

User tiếp tục: {new_message}

Hãy tiếp nối câu chuyện một cách tự nhiên, sử dụng context ở trên.
"""

        # Gọi LLM
        response = self.llm.invoke(prompt)

        # Lưu message mới
        self.save_conversation(user_id, "user", new_message, session_id)
        self.save_conversation(user_id, "assistant", response.content, session_id)

        # Cập nhật context
        context['last_interaction'] = datetime.now().isoformat()
        self.save_conversation_context(user_id, session_id, context)

        return response.content

    def _build_conversation_context(self, history: List[Dict], context: Dict) -> str:
        """Xây dựng context từ history và saved context"""
        context_str = "CONTEXT ĐỂ TIẾP NỐI CUỘC TRÒ CHUYỆN:\n\n"

        # Thêm context đã lưu
        if context:
            context_str += "Thông tin từ phiên trước:\n"
            for key, value in context.items():
                if key not in ['last_interaction']:
                    context_str += f"- {key}: {value}\n"
            context_str += "\n"

        # Thêm lịch sử hội thoại
        if history:
            context_str += "Lịch sử trò chuyện gần nhất:\n"
            for msg in history[-5:]:  # Lấy 5 tin nhắn gần nhất
                context_str += f"{msg['role']}: {msg['content']}\n"

        return context_str

    def create_new_session(self, user_id: str, initial_context: Dict = None) -> str:
        """Tạo session mới"""
        session_id = str(uuid.uuid4())
        context = initial_context or {
            "created_at": datetime.now().isoformat(),
            "user_id": user_id,
            "status": "active"
        }

        self.save_conversation_context(user_id, session_id, context)
        return session_id

# 2. HỆ THỐNG TƯƠNG TÁC ĐỂ TIẾP NỐI CHAT
class InteractiveChatContinuer:
    def __init__(self):
        self.chat_system = ChatContinuationSystem()
        self.current_user = "default_user"
        self.current_session = None

    def start_interactive_mode(self):
        """Chế độ tương tác để tiếp nối chat"""
        print("🤖 CHAT CONTINUATION SYSTEM")
        print("=" * 60)

        # Kiểm tra user có sessions nào không
        sessions = self.chat_system.get_user_sessions(self.current_user)

        if sessions:
            print("📚 Tìm thấy các phiên chat cũ:")
            for i, session in enumerate(sessions, 1):
                print(f"{i}. Session: {session['session_id'][:8]}...")
                print(f"   Last updated: {session['last_updated'][:16]}")
                if 'current_step' in session['context']:
                    print(f"   Đang ở: {session['context']['current_step']}")
                print()

            choice = input("Chọn phiên để tiếp tục (số), hoặc 'new' để tạo mới: ").strip()

            if choice.lower() == 'new':
                self._create_new_session()
            elif choice.isdigit() and 1 <= int(choice) <= len(sessions):
                selected_session = sessions[int(choice) - 1]
                self.current_session = selected_session['session_id']
                print(f"✅ Đã chọn phiên: {self.current_session[:8]}...")
                self._continue_chat_session()
            else:
                print("Tạo phiên mới...")
                self._create_new_session()
        else:
            print("Chưa có phiên chat nào. Tạo mới...")
            self._create_new_session()

    def _create_new_session(self):
        """Tạo session chat mới"""
        self.current_session = self.chat_system.create_new_session(self.current_user)
        print(f"✅ Đã tạo phiên mới: {self.current_session[:8]}...")
        print("Bắt đầu chat mới!")
        self._chat_loop()

    def _continue_chat_session(self):
        """Tiếp tục chat session đã chọn"""
        # Load và hiển thị context cũ
        context = self.chat_system.load_conversation_context(self.current_user, self.current_session)
        history = self.chat_system.get_conversation_history(self.current_user, self.current_session, 5)

        print("\n📋 Context từ phiên trước:")
        for key, value in context.items():
            if key not in ['created_at', 'last_interaction']:
                print(f"   {key}: {value}")

        print("\n💬 Lịch sử gần nhất:")
        for msg in history:
            print(f"   {msg['role']}: {msg['content'][:50]}...")

        print("\n🔮 Bạn có thể tiếp tục ngay!")
        self._chat_loop()

    def _chat_loop(self):
        """Vòng lặp chat chính"""
        print("\n💬 Nhập tin nhắn của bạn (hoặc 'quit' để thoát):")

        while True:
            try:
                user_input = input("\n👤 Bạn: ").strip()

                if user_input.lower() in ['quit', 'exit', 'thoát']:
                    print("👋 Kết thúc chat!")
                    break

                if not user_input:
                    continue

                # Tiếp tục conversation
                response = self.chat_system.continue_conversation(
                    self.current_user,
                    self.current_session,
                    user_input
                )

                print(f"🤖 Agent: {response}")

            except KeyboardInterrupt:
                print("\n👋 Tạm dừng chat!")
                break
            except Exception as e:
                print(f"❌ Lỗi: {e}")
                break

# 4. MAIN EXECUTION
if __name__ == "__main__":
    # Tạo dữ liệu mẫu
    demo_session_id = demo_with_sample_data()

    # Khởi tạo và chạy interactive continuer
    continuer = InteractiveChatContinuer()
    continuer.current_user = "demo_user"
    continuer.current_session = demo_session_id

    # Chạy continuation
    continuer._continue_chat_session()

c:\Users\minhv\anaconda3\envs\pytorch-env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


🧪 TẠO DỮ LIỆU MẪU...
✅ Đã tạo session mẫu: 6754d615...
💬 Conversation history đã được lưu!


ValueError: Expected where to have exactly one operator, got {'user_id': 'demo_user', 'session_id': '6754d615-13d8-47b5-ae50-8b545077cc10'} in get.

In [ ]:
from memory.memory import memory_manager

session_id = "session_20250909_164021"  # id bạn muốn tiếp tục
n_loaded = memory_manager.resume_session(session_id, user_id="default_user", replay_last_n=20)
print(f"Resumed {n_loaded} messages from long-term for session {session_id}")

run_multi_agent_chat()  # dùng loop chat hiện tại của bạn

Resumed 17 messages from long-term for session session_20250909_164021
🤖 Multi-Agent with Short-Term & Long-Term Memory
Commands: 'exit', 'clear' (short-term), 'clear_all' (both), 'mem_stats'

🤖travel_agent: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
   [Memory: 18 msgs, 0 prefs]


KeyboardInterrupt: Interrupted by user

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from Agents_2.Coordinator_Agent import coordinator_agent
from Agents_2.Travel_Agent import travel_agent
from Agents_2.Hotel_Agent import hotel_agent
from Agents_2.Flight_Agent import flight_agent
from memory.memory import memory_manager

def create_initial_state():
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }

AGENT_MAP = {
    "coordinator": coordinator_agent,
    "travel_agent": travel_agent,
    "hotel_agent": hotel_agent,
    "flight_agent": flight_agent,
}

def run_multi_agent_chat():
    print("🤖 Multi-Agent with Short-Term & Long-Term Memory")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'")

    state = create_initial_state()

    while True:
        if not state["needs_user_input"]:
            step_fn = AGENT_MAP.get(state["current_agent"], coordinator_agent)
            state = step_fn(state)

            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")

            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Long-term memory has been saved.")
            break

        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term vẫn giữ.")
            continue

        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue

        if user_input.lower() in ["mem_stats", "memory_stats"]:
            from memory.long_term_memory import long_term_memory
            stats = long_term_memory.collection.count()
            print(f"📊 Long-term Memory: {stats} items stored")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        state["needs_user_input"] = False

In [ ]:
from memory.memory import memory_manager

def continue_chat_from_session(session_id: str, user_id: str = "default_user", replay_last_n: int = 20):
    loaded = memory_manager.resume_session(session_id, user_id=user_id, replay_last_n=replay_last_n)
    print(f"Resumed {loaded} messages from long-term: {session_id}")
    run_multi_agent_chat()

# Ví dụ gọi:
continue_chat_from_session("session_20250909_164021")

Resumed 18 messages from long-term: session_20250909_164021
🤖 Multi-Agent with Short-Term & Long-Term Memory
Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'

🤖travel_agent: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
   [Memory: 37 msgs, 0 prefs]

🤖coordinator: Thân ái! Bạn đã quyết định đi du lịch rồi!

Để tạo một chuyến đi hấp dẫn, tôi sẽ cần biết thêm về sở thích và nhu cầu của bạn. Xin hãy chia sẻ với tôi:

1. Điểm đến mong muốn?
2. Thời gian đi du lịch là bao lâu?
3. Bạn có ai trong gia đình đi cùng hay không?

Sau khi tôi có thông tin này, tôi sẽ tạo một lịch trình du lịch sáng tạo, hấp dẫn và đề xuất điểm đến, hoạt động, ẩm thực phù hợp với sở thích của bạn.

Và sau đó, tôi sẽ giúp bạn đặt vé máy bay và khách sạn!

Xin chào!
   [Memory: 39 msgs, 0 prefs]
👋 Bye. Long-term memory has been saved.


----------------------------------------TEST MODEL LLM-----------------------------------------

In [ ]:
import os
from langchain_community.chat_models import ChatOllama
from IPython.display import display, Markdown
from dotenv import load_dotenv

load_dotenv()

llm = ChatOllama(model="gpt-oss")   # model đã có trong Ollama

prompt = "Trường đại học Khoa học Tự nhiên TP HCM có bao nhiêu ngành"

response = llm.invoke(prompt)

display(Markdown(response.content))


KeyboardInterrupt: 

----------------------------------------TEST MODEL LLM-----------------------------------------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 25.4 MB/s eta 0:00:00


In [7]:

import easyocr

def image_to_text_easyocr(image_path):
    reader = easyocr.Reader(['en','vi'])  # hỗ trợ tiếng Anh + tiếng Việt
    result = reader.readtext(image_path, detail=0)  # detail=0 để chỉ lấy text
    return "\n".join(result)

image_path = "./drive/MyDrive/images/Screenshot 2025-10-04 154203.png"
text = image_to_text_easyocr(image_path)

# Export
with open("./drive/MyDrive/export_to_text/text1.txt", "w", encoding="utf-8") as f:
    f.write(text)
print("✅ OCR xong và đã lưu file")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ OCR xong và đã lưu file
